In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
import itertools

import warnings
warnings.filterwarnings("ignore")

In [ ]:
data=pd.read_csv("Company Stock and Investment.csv")
data.head()

In [ ]:
data.info()

In [ ]:
data['Date']=pd.to_datetime(data['Date'],format='%d-%m-%Y')

# Gold Investments


In [ ]:
#Lets consider the Gold Investments as our target and drop all others columns.

df=data.drop(columns=["Oil Investments","Comp Stock","Other sharesInvestments"],axis=1)
df.head()

In [ ]:
# making date as my index column

df.set_index('Date',inplace=True)
df.head()

In [ ]:
df.plot()

In [ ]:
df.describe()

In [ ]:
#check if data is stationarity or not

def adfuller_test(target):
    result=adfuller(target)
    labels = ['ADF Test Statistic','p-value','#Lags Used','Number of Observations Used']
    for value,label in zip(result,labels):
        print(label+' : '+str(value) )
    if result[1] <= 0.05:
        print("strong evidence against the null hypothesis(Ho), reject the null hypothesis. Data  is stationary")
    else:
        print("weak evidence against null hypothesis, time series is non-stationary ")

In [ ]:
adfuller_test(df['Gold Investments'])

In [ ]:
# average the daily stock value of gold investment for each month

y = df['Gold Investments'].resample('MS').mean()
y['2009':]

In [ ]:
y.plot(figsize = (15, 6))

In [ ]:
from pylab import rcParams
rcParams['figure.figsize'] = 15, 8

decomposition = sm.tsa.seasonal_decompose(y, model = 'additive')
fig = decomposition.plot()
plt.show()

Visualizing our data using a method called time-series decomposition that allows us to decompose our time series into three distinct components: trend, seasonality, and noise.

# Time series forecasting with ARIMA
Applying one of the most commonly used method for time-series forecasting, known as ARIMA, which stands for Autoregressive Integrated Moving Average.ARIMA models are denoted with the notation ARIMA(p, d, q). These three parameters account for seasonality, trend, and noise in data.

In [ ]:
# set the typical ranges for p, d, q
p = d = q = range(0, 2)

pdq = list(itertools.product(p, d, q))
seasonal_pdq = [(x[0], x[1], x[2], 12) for x in list(itertools.product(p, d, q))]

print('Examples of parameter combinations for Seasonal ARIMA...')
print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[1]))
print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[2]))
print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[3]))
print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[4]))

In [ ]:
# Using Grid Search find the optimal set of parameters that yields the best performance
for param in pdq:
    for param_seasonal in seasonal_pdq:
        try:
            mod = sm.tsa.statespace.SARIMAX(y, order = param, seasonal_order = param_seasonal, enforce_stationary = False,enforce_invertibility=False) 
            result = mod.fit()   
            print('ARIMA{}x{}12 - AIC:{}'.format(param, param_seasonal, result.aic))
        except:
            continue

The above output suggests that SARIMAX(1, 0, 0)x(1, 0, 1, 12) yields the lowest AIC value of -482.2437. Therefore we should consider this to be optimal option.

In [ ]:
model = sm.tsa.statespace.SARIMAX(y, order = (1, 0, 0),seasonal_order = (1, 0, 1, 12))
result = model.fit()
print(result.summary().tables[1])

In [ ]:
prediction = result.get_prediction(start = pd.to_datetime('2016-07-01'), dynamic = False)
prediction_ci = prediction.conf_int()
prediction_ci

# Validating forecasts
To help us understand the accuracy of our forecasts, we compare predicted stock value to real stock value of the time series, and we set forecasts to start at 2016–07–01 to the end of the data

In [ ]:
pred = result.get_prediction(start=pd.to_datetime('2016-07-01'), dynamic=False)
pred_ci = pred.conf_int()
ax = y['2015':].plot(label='observed Stock')
pred.predicted_mean.plot(ax=ax, label='Forecast stock', alpha=.7, figsize=(14, 7))
ax.fill_between(pred_ci.index,
                pred_ci.iloc[:, 0],
                pred_ci.iloc[:, 1], color='k', alpha=.2)
ax.set_xlabel('Date')
ax.set_ylabel('Gold Investments')
plt.legend()
plt.show()


In [ ]:
y_forecasted = pred.predicted_mean
y_truth = y['2016-07-01':]
mse = ((y_forecasted - y_truth) ** 2).mean()
print('The Mean Squared Error of our forecasts is {}'.format(round(mse, 2)))

In [ ]:
print('The Root Mean Squared Error of our forecasts is {}'.format(round(np.sqrt(mse), 2)))

In [ ]:
pred_uc = result.get_forecast(steps=50)
pred_ci = pred_uc.conf_int()
ax = y.plot(label='observed', figsize=(14, 7))
pred_uc.predicted_mean.plot(ax=ax, label='Forecast')
ax.fill_between(pred_ci.index,
                pred_ci.iloc[:, 0],
                pred_ci.iloc[:, 1], color='k', alpha=.25)
ax.set_xlabel('Date')
ax.set_ylabel('Gold Investments')
plt.legend()
plt.show()